In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import re
import os

In [4]:
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
df_train = pd.read_csv("data/train.csv")

df_test = pd.read_csv("data/test.csv")

print(df_test.head())

def format_row_as_string(row):
    row_string = f"Tweet ID: {row['id']}, "
    row_string += f"Keyword: {row['keyword'] if pd.notnull(row['keyword']) else 'N/A'}, "
    row_string += f"Location: {row['location'] if pd.notnull(row['location']) else 'N/A'}, "
    row_string += f"Text: {row['text']}."
    return row_string


   id keyword location                                               text
0   0     NaN      NaN                 Just happened a terrible car crash
1   2     NaN      NaN  Heard about #earthquake is different cities, s...
2   3     NaN      NaN  there is a forest fire at spot pond, geese are...
3   9     NaN      NaN           Apocalypse lighting. #Spokane #wildfires
4  11     NaN      NaN      Typhoon Soudelor kills 28 in China and Taiwan


In [13]:
def query(info):
    with open('prompt2.txt', 'r') as file:
        prompt_instructions = file.read()

    messages = [
        {"role": "user", "content": prompt_instructions},
        {"role": "assistant", "content": "CONFIRM"},
        {"role": "user", "content": info}
    ]

    print(messages)
    encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt", padding=True, return_attention_mask=True)

    model_inputs = encodeds.to(device)
    model.to(device)

    generated_ids = model.generate(model_inputs, max_new_tokens=5, do_sample=True, pad_token_id=tokenizer.eos_token_id)
    decoded = tokenizer.batch_decode(generated_ids)
    return decoded[0]

In [27]:
temp_folder = "data/temp/"

# Function to process and save batches of DataFrame rows
def process_batch(df_batch, start_index):
    responses = []
    df_length = df_batch.shape[0]
    for index, row in tqdm(df_batch.iterrows(), total=df_length, position=1, desc="Inner", leave=False):
        info = format_row_as_string(row)
        response = query(info)
        # Reverse the response to deal with it from the end
        reversed_response = response[::-1]
        # Find the first digit in the reversed response
        match = re.search(r'\d', reversed_response)
        if match:
            # Get the matched digit
            digit = match.group(0)
            classification = digit if digit in ['0', '1'] else "Unknown"
        else:
            classification = "Unknown"
        responses.append({'id': row['id'], 'target': classification})
    # Convert to DataFrame and save as CSV
    batch_df = pd.DataFrame(responses)
    batch_df.to_csv(f"{temp_folder}batch_{start_index}.csv", index=False)

# Function to process the DataFrame in chunks and save the results
def process_and_save(df, num_rows=-1):
    if num_rows == -1:
        num_rows = len(df)
    for start in tqdm(range(0, num_rows, 10), position=0, desc="Outer"):
        end = min(start + 10, num_rows)
        df_batch = df.iloc[start:end]
        process_batch(df_batch, start)

# Function to combine all CSV files from the temp folder into one DataFrame
def combine_csv(temp_folder):
    all_files = [os.path.join(temp_folder, f) for f in os.listdir(temp_folder) if f.endswith('.csv')]
    df_list = [pd.read_csv(file) for file in all_files]
    combined_df = pd.concat(df_list, ignore_index=True)
    return combined_df

process_and_save(df_test, num_rows=-1)

# Combine all the CSV files after processing is done
final_df = combine_csv(temp_folder)
final_df.to_csv("output/final_llm_classification.csv", index=False)


Outer:   0%|          | 0/327 [00:00<?, ?it/s]

Inner:   0%|          | 0/10 [00:00<?, ?it/s]

Inner:   0%|          | 0/10 [00:00<?, ?it/s]

Inner:   0%|          | 0/10 [00:00<?, ?it/s]

Inner:   0%|          | 0/10 [00:00<?, ?it/s]

...

Inner:   0%|          | 0/10 [00:00<?, ?it/s]

Inner:   0%|          | 0/10 [00:00<?, ?it/s]

Inner:   0%|          | 0/10 [00:00<?, ?it/s]

Inner:   0%|          | 0/10 [00:00<?, ?it/s]

Inner:   0%|          | 0/3 [00:00<?, ?it/s]

In [24]:
df_predictions = pd.read_csv("output/final_llm_classification.csv")

df_merged = df_train.merge(df_predictions, on='id', suffixes=('_actual', '_predicted'))
df_merged['incorrect_prediction'] = ((df_merged['target_actual'] != df_merged['target_predicted']) &
                                     (df_merged['target_predicted'] != -1))

# Calculate metrics
accuracy = accuracy_score(df_merged['target_actual'], df_merged['target_predicted'])
precision = precision_score(df_merged['target_actual'], df_merged['target_predicted'], zero_division=0)
recall = recall_score(df_merged['target_actual'], df_merged['target_predicted'])
f1 = f1_score(df_merged['target_actual'], df_merged['target_predicted'])

# Output the metrics
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

incorrect_predictions = df_merged[df_merged['incorrect_prediction']]
incorrect_predictions.to_csv("output/incorrect_predictions.csv", index=False)


Accuracy: 0.96
Precision: 1.0
Recall: 0.9
F1 Score: 0.9473684210526315


Given the tweet text provided by the user, determine whether it represents a real disaster (actual events like earthquakes, floods, fires, etc.) or not. Your response should be a single number: 1 if the tweet is about a real disaster, and 0 if it is not about a real disaster. Consider the context, wording, and any potential for metaphorical language that might indicate the tweet is not referring to an actual disaster. Please analyze the tweet carefully and provide your classification based on the content.

Instructions to Language Model:
Review the tweet carefully.
Consider the use of specific disaster-related keywords and assess whether they are used to describe real events or metaphorically.
Ensure no additional text, explanation, or content is added beyond this number.

Reply CONFIRM if you understand.

Accuracy: 0.94
Precision: 1.0
Recall: 0.85
F1 Score: 0.918918918918919

Given the tweet text provided, determine whether it represents a real disaster (actual events like earthquakes, floods, fires, etc.) or not. Analyze the context deeply, focusing on specific disaster-related keywords, mentions of locations, any temporal references, and the overall tone. Consider the broader situational context and subtleties in language use. In cases of ambiguity, where the tweet could reasonably be about a real disaster based on the context and use of language, classify as a real disaster. Your response should be a single number: 1 if the tweet is about a real disaster, and 0 if it is not.

Accuracy: 0.96
Precision: 1.0
Recall: 0.9
F1 Score: 0.9473684210526315